In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import re
import torch.nn as nn
import torch
from bayesian_nnet.data import IQDataModel

In [3]:
iqdata = IQDataModel()
iqdata.setup(stage="")
dl = iqdata.train_dataloader()
batch, _, _ = dl.__iter__().__next__()
batch = batch[:10, :, :]

In [4]:
baud_rates = pd.read_csv("./baudrate.csv")
po = re.compile(r"^[0-9]+$")
h_is_numeric = lambda elem: po.match(elem) is not None
select_row = baud_rates["Baud Rate"].apply(h_is_numeric)
baud_rates = baud_rates.loc[select_row, :]
6000 / baud_rates["Baud Rate"].astype(float) / 64

1     3.024194
2     1.488095
3     3.024194
4     2.083333
5     1.875000
6     0.937500
7     3.024194
8     3.024194
9     1.512097
10    3.024194
11    8.522727
12    9.375000
13    0.937500
Name: Baud Rate, dtype: float64

In [5]:
"""
Layer #1
---------
conv 3, 64
ReLU
max pool / 2
"""
conv1 =\
    nn.Sequential(
        nn.Conv1d(in_channels=2,
                  out_channels=64,
                  kernel_size=3,
                  padding=1),
        nn.BatchNorm1d(64),
        nn.ReLU(),
        nn.MaxPool1d(kernel_size=3,
                     stride=2,
                     padding=1))

"""
Layer #2
---------
conv 3, 64
ReLU
max pool / 2
"""
conv2 =\
    nn.Sequential(
        nn.Conv1d(in_channels=64,
                  out_channels=64,
                  kernel_size=3,
                  padding=1),
        nn.BatchNorm1d(64),
        nn.ReLU(),
        nn.MaxPool1d(kernel_size=3,
                     stride=2,
                     padding=1))

"""
Layer #3
---------
conv 3, 128
ReLU
max pool / 2
"""
conv3 =\
    nn.Sequential(
        nn.Conv1d(in_channels=64,
                  out_channels=128,
                  kernel_size=3,
                  padding=1),
        nn.BatchNorm1d(128),
        nn.ReLU(),
        nn.MaxPool1d(kernel_size=3,
                     stride=2,
                     padding=1))

"""
Layer #4
---------
conv 3, 128
ReLU
max pool / 2
"""
conv4 =\
    nn.Sequential(
        nn.Conv1d(in_channels=128,
                  out_channels=128,
                  kernel_size=3,
                  padding=1),
        nn.BatchNorm1d(128),
        nn.ReLU(),
        nn.MaxPool1d(kernel_size=3,
                     stride=2,
                     padding=1))

"""
Layer #5
---------
conv 3, 256
ReLU
max pool / 2
"""
conv5 =\
    nn.Sequential(
        nn.Conv1d(in_channels=128,
                  out_channels=256,
                  kernel_size=3,
                  padding=1),
        nn.BatchNorm1d(256),
        nn.ReLU(),
        nn.MaxPool1d(kernel_size=3,
                     stride=2,
                     padding=1))

"""
Layer #6
---------
conv 3, 256
ReLU
max pool / 2
"""
conv6 =\
    nn.Sequential(
        nn.Conv1d(in_channels=256,
                  out_channels=256,
                  kernel_size=3,
                  padding=1),
        nn.BatchNorm1d(256),
        nn.ReLU(),
        nn.MaxPool1d(kernel_size=3,
                     stride=2,
                     padding=1),
        nn.Dropout1d(0.6, inplace=True))

In [8]:
layer_out = conv1(batch)
print(layer_out.shape)
layer_out = conv2(layer_out)
print(layer_out.shape)
layer_out = conv3(layer_out)
print(layer_out.shape)
layer_out = conv4(layer_out)
print(layer_out.shape)
layer_out = conv5(layer_out)
print(layer_out.shape)
layer_out = conv6(layer_out)
print(layer_out.shape)

torch.Size([10, 64, 1024])
torch.Size([10, 64, 512])
torch.Size([10, 128, 256])
torch.Size([10, 128, 128])
torch.Size([10, 256, 64])
torch.Size([10, 256, 32])


In [11]:
lstm = nn.LSTM(input_size=256,
               hidden_size=64,
               num_layers=2,
               batch_first=True)

In [12]:
layer_out = torch.permute(layer_out, (0, 2, 1))
layer_out, _ = lstm(layer_out)
layer_out = torch.permute(layer_out, (0, 2, 1))
print(layer_out.shape)
layer_out = torch.flatten(layer_out, start_dim=1)
print(layer_out.shape)

torch.Size([10, 64, 32])
torch.Size([10, 2048])


In [29]:
fc_layers =\
    nn.Sequential(nn.Linear(2048, 128),
                  nn.Dropout(0.5),
                  nn.BatchNorm1d(128),
                  nn.ReLU(),
                  nn.Linear(128, 18))

fc_layers(layer_out).shape

torch.Size([10, 18])

In [15]:
from bayesian_nnet.cnnlstm import IQDataEmbedder, FrequentistClassifier
iq_embed = IQDataEmbedder()
mod_clf = FrequentistClassifier()
layer_out = iq_embed(batch)
print(layer_out.shape)
mod_clf(layer_out).shape

torch.Size([10, 2048])


torch.Size([10, 18])